# Training MusicLM model
### Robert Chen, Ahmadsho Akdodshoev, Philip Timofeev

## 0. Imports

In [3]:
!pip install musiclm-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.1/819.1 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 58.1 MB/s eta 0:00:00:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 51.3 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchaudio
from musiclm_pytorch import MuLaN, MuLaNEmbedQuantizer, MuLaNTrainer, \
                            AudioSpectrogramTransformer, TextTransformer, MusicLM
from audiolm_pytorch import SemanticTransformer, SemanticTransformerTrainer, \
                            CoarseTransformer, CoarseTransformerTrainer, \
                            FineTransformer, FineTransformerTrainer, \
                            AudioLM, HubertWithKmeans, MusicLMSoundStream, \
                            SoundStreamTrainer, SoundStream 
import os
import wave
import urllib.request

## 1. Creating dataloaders and downloading Hubert K-means checkpoints

Creating the dataset

In [5]:
dataset_path = '/kaggle/input/soul-mzk/'

class MusicLMDataset(Dataset):
    def __init__(self, path: str):
        pass
    def __getitem__(self, idx):
        pass
    
train_dataset = MusicLMDataset(dataset_path)

Downloading Hubert checkpoints

In [6]:
hubert_ckpt = 'hubert/hubert_base_ls960.pt'
hubert_quantizer = 'hubert/hubert_base_ls960_L9_km500.bin'
soundstream_ckpt = './results/soundstream.pt'

if not os.path.isdir("hubert"):
  os.makedirs("hubert")
if not os.path.isfile(hubert_ckpt):
  hubert_ckpt_download = f"https://dl.fbaipublicfiles.com/{hubert_ckpt}"
  urllib.request.urlretrieve(hubert_ckpt_download, f"./{hubert_ckpt}")
if not os.path.isfile(hubert_quantizer):
  hubert_quantizer_download = f"https://dl.fbaipublicfiles.com/{hubert_quantizer}"
  urllib.request.urlretrieve(hubert_quantizer_download, f"./{hubert_quantizer}")

## 2. Training MuLaN

Arguments for every module are defined in the respective dictionaries to make fine-tuning easier

In [7]:
AUDIO_KWARGS = {
    'dim': 512,
    'depth': 6,
    'heads': 8,
    'dim_head': 64,
    'spec_n_fft': 128,
    'spec_win_length': 24,
    'spec_aug_stretch_factor': 0.8
}

TEXT_KWARGS = {
    'dim': 512,
    'depth': 6,
    'heads': 8,
    'dim_head': 64
}

MULAN_KWARGS = {
    'dataset': train_dataset
    'num_train_steps': 10,
    'batch_size': 4,
    'force_clear_prev_results': True,
    'save_model_every': 5
}

MULAN_QUANTIZER_KWARGS = {
    'conditioning_dims': (1024, 1024, 1024),
    'namespaces': ('semantic', 'coarse', 'fine')
}

HUBERT_KWARGS = {
    'checkpoint_path': hubert_ckpt,
    'kmeans_path': hubert_quantizer
}

SOUNDSTREAM_TRAINER_KWARGS = {
    'folder': dataset_path,
    'num_train_steps': 20,
    'save_model_every': 2,
    'batch_size': 4
}
    
SEMANTIC_KWARGS = {
    'dim': 1024,
    'depth': 6,
    'audio_text_condition': True 
}

COARSE_KWARGS = {
    'codebook_size': 1024,
    'num_coarse_quantizers': 4,
    'dim': 1024,
    'depth': 6,
    'audio_text_condition': True 
}

FINE_KWARGS = {
    'codebook_size': 1024,
    'num_coarse_quantizers': 4,
    'num_fine_quantizers': 8,
    'dim': 1024,
    'depth': 6,
    'audio_text_condition': True 
}

TRANSFORMER_TRAINER_KWARGS = {
    'folder': dataset_path,
    'num_train_steps': 10,
    'save_model_every': 2,
    'batch_size': 4,
    'data_max_length': 320 * 32
}

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

Training MuLaN

In [8]:
audio_transformer = AudioSpectrogramTransformer(**AUDIO_KWARGS)

text_transformer = TextTransformer(**TEXT_KWARGS)

mulan = MuLaN(audio_transformer, text_transformer)

mulan_trainer = MuLaNTrainer(mulan, **MULAN_KWARGS)

mulan_trainer.train()

mulan_trainer.save()

TypeError: MuLaNTrainer.__init__() missing 1 required positional argument: 'dataset'

## 3. Training SoundStream

In [ ]:
soundstream = MusicLMSoundStream()
soundstream_trainer = SoundStreamTrainer(
    soundstream,
    **SOUNDSTREAM_TRAINER_KWARGS
)

soundstream_trainer.train()

soundstream_trainer.save(soundstream_ckpt)

## 4. Training conditioning embeddings

Defining the MuLaN Embed Quantizer and Hubert K-means Embedder

In [ ]:
quantizer = MuLaNEmbedQuantizer(
    mulan=mulan,                         
    **MULAN_QUANTIZER_KWARGS
)

wav2vec = HubertWithKmeans(
    **HUBERT_KWARGS
)

Training Semantic Transformer

In [ ]:
semantic_transformer = SemanticTransformer(
   num_semantic_tokens=wav2vec.codebook_size,
   **SEMANTIC_KWARGS 
).to(DEVICE)

semantic_trainer = SemanticTransformerTrainer(
    wav2vec,
    semantic_transformer,
    audio_conditioner=quantizer,
    **TRANSFORMER_TRAINER_KWARGS
)

semantic_trainer.train()

Training Coarse Transformer

In [ ]:
soundstream = MusicLMSoundStream()

soundstream.load(soundstream_ckpt)

coarse_transformer = CoarseTransformer(
    num_semantic_tokens=wav2vec.codebook_size,
    **COARSE_KWARGS
).to(DEVICE)

coarse_trainer = CoarseTransformerTrainer(
    wav2vec,
    semantic_transformer,
    codec=soundstream,
    audio_conditioner=quantizer,
    **TRANSFORMER_TRAINER_KWARGS
)

coarse_trainer.train()

Training Fine Transformer

In [ ]:
soundstream = MusicLMSoundStream()

soundstream.load(soundstream_ckpt)

fine_transformer = FineTransformer(
    codebook_size=wav2vec.codebook_size,
    **FINE_KWARGS
).to(DEVICE)

fine_trainer = FineTransformerTrainer(
    wav2vec,
    semantic_transformer,
    codec=soundstream
    audio_conditioner=quantizer,
    **TRANSFORMER_TRAINER_KWARGS
)

fine_trainer.train()

## 5. Combining AudioLM and MusicLM

In [ ]:
audio_lm = AudioLM(
    wav2vec=wav2vec,
    codec=soundstream,
    semantic_transformer=semantic_transformer,
    coarse_transformer=coarse_transformer,
    fine_transformer=fine_transformer   
)

In [ ]:
music_lm = MusicLM(
    audio_lm=audio_lm,
    mulan_embed_quantizer=quantizer
)

music = music_lm('soul', num_samples=1)

In [ ]:
torch.save(music, 'generated_music.pt')

In [ ]:
output_path = "out.wav"
sample_rate = 44100
torchaudio.save(output_path, music.cpu(), sample_rate)